# Assignment #2 Regression

Submitted by:

Jazib Imran

Anjali Bhagat

Adopted from: https://geemap.org/notebooks/32_supervised_classification/

In [2]:
import ee
import geemap

In [3]:
# If you are running this notebook for the first time, you need to activate the command below for the authentication flow:
ee.Authenticate()

True

In [4]:
try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


For this regression task, we would use Nairobi as our Region of interest.
We take the central point of Nairobi and then get its 10000m bounds to run our classfication problem

In [8]:
Map= geemap.Map()
point = ee.Geometry.Point([36.8219, -1.2921])
boundingBox = point.buffer(10000).bounds()

# Display the bounding box
Map.addLayer(boundingBox, {}, "Nairobi Bounding Box")
Map.centerObject(point, 10)
Map

Map(center=[-1.2921, 36.8219], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

Collecting the Landsat-8 data

In [17]:
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2020-01-01', '2020-12-31') \
    .filterBounds(boundingBox)

We can also print how many images are there in our collection

In [18]:
listOfImages = landsat8.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', len(listOfImages))

Number of images in the collection:  22


### NDVI Calculation and Monthly Composite Generation

To calculate monthly NDVI composites for the year 2020, we begin by defining a function to compute NDVI using the near-infrared (NIR) and red bands (`SR_B5` and `SR_B4`, respectively) from the Landsat 8 surface reflectance data. The `calculate_ndvi` function performs this calculation for each image in the Landsat 8 collection by applying the `normalizedDifference` function, which is typical for NDVI calculation:

1. **NDVI Calculation**: The NIR and red bands are combined to calculate NDVI, which is added as a new band to each image in the collection.
  
2. **Monthly Composites**: We define a `monthly_composite` function that creates a monthly average NDVI composite. For each month in 2020:
   - The function filters the collection to include images within that month.
   - It calculates the mean NDVI and adds the month as metadata.
  
3. **Generate Collection for 2020**: Using a mapped function, the code iterates over each month (from January to December) to create an image collection containing monthly NDVI composites for the entire year.


In [ ]:
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')  # B5=NIR, B4=Red
    return image.addBands(ndvi)
landsat8_ndvi = landsat8.map(calculate_ndvi)
def monthly_composite(month):
    start_date = ee.Date.fromYMD(2020, month, 1)
    end_date = start_date.advance(1, 'month')
    monthly_ndvi = landsat8_ndvi.filterDate(start_date, end_date).select('NDVI')
    return monthly_ndvi.mean().set('month', month)  # Create monthly composite and set metadata

months = ee.List.sequence(1, 12)
monthly_ndvi_2020 = ee.ImageCollection(months.map(lambda month: monthly_composite(ee.Number(month))))

### Precipitation and Elevation Data Extraction

This code snippet extracts precipitation and elevation data for the year 2020:

1. **Precipitation Data**: It loads daily precipitation data from the CHIRPS dataset, filtering it to include only the year 2020 and the specified bounding box. The total annual precipitation is then calculated by summing the daily values.

2. **Elevation Data**: The SRTM elevation data is retrieved and clipped to the same bounding box.

3. **NDVI Mean Calculation**: The mean NDVI is computed from the previously generated monthly NDVI composites for the year.


In [ ]:
precipitation = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
    .filterDate('2020-01-01', '2020-12-31') \
    .filterBounds(boundingBox)
elevation = ee.Image('USGS/SRTMGL1_003').clip(boundingBox)
annual_precipitation_2020 = precipitation.sum().clip(boundingBox)
ndvi_mean=monthly_ndvi_2020.mean()

In [41]:
# Stack predictor and target variables into one image for sampling
stacked_image = elevation.rename('elevation') \
    .addBands(annual_precipitation_2020.rename('precipitation')) \
    .addBands(ndvi_mean.rename('NDVI'))

### Random Sampling of Predictor and Target Variables

This code snippet demonstrates how to sample predictor variables (elevation and precipitation) and the target variable (NDVI) using random points:

1. **Random Points Generation**: 1000 random sampling points are generated within the specified bounding box. A seed is set for reproducibility.

2. **Data Sampling**: The `sampleRegions` function is utilized on the stacked image containing NDVI, elevation, and precipitation data. The sampling is conducted at a scale of 30 meters (the resolution of Landsat data), and geometries are retained to identify the locations of the samples.


In [42]:
random_points = ee.FeatureCollection.randomPoints(boundingBox, 1000, seed=42)
sampled_data = stacked_image.sampleRegions(
    collection=random_points,
    scale=30,  # Scale set to Landsat resolution
    geometries=True  # Retain geometries to know where each sample was taken
)

Now we conver that samples_data into a dataframe

In [44]:
df=geemap.ee_to_df(sampled_data)
df

,NDVI,elevation,precipitation
0,0.166124,1589,1070.962303
1,0.174285,1713,1070.547753
2,0.112632,1644,1048.908933
3,0.092606,1618,1048.908933
4,0.077691,1645,988.865744
...,...,...,...
995,0.195149,1634,1064.276649
996,0.054047,1597,1048.908933
997,0.181331,1660,1070.547753
998,0.142138,1876,1314.800226


### Machine Learning Regression Analysis

In this code snippet, we perform a regression analysis using scikit-learn to predict NDVI based on elevation and precipitation:

1. **Library Imports**: Necessary libraries such as `pandas`, `sklearn`, and `numpy` are imported for data manipulation, model training, and evaluation.

2. **Define Predictors and Target**: The predictors (`elevation` and `precipitation`) and the target variable (`NDVI`) are defined from the DataFrame `df`.

3. **Data Splitting**: The dataset is split into training and testing sets using an 80-20 split, ensuring a random state for reproducibility.

4. **Model Initialization**: A linear regression model is initialized.

5. **Model Training**: The model is fitted to the training data.

6. **Predictions**: Predictions are made on the testing set.

7. **Performance Evaluation**: The model's performance is evaluated using Mean Squared Error (MSE) and R-squared (R²) metrics, which are printed to the console.


In [47]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

X = df[['elevation', 'precipitation']]
y = df['NDVI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 Score): {r2}")



Mean Squared Error (MSE): 0.003076259303782547
R-squared (R2 Score): 0.09501113162379393


# Machine Learning Regression Analysis with Dask

In this section, we implement a regression analysis using Dask to predict NDVI based on elevation and precipitation. Dask is utilized for handling larger datasets that do not fit into memory and enables parallel processing.

## Steps:

1. **Convert Data to Dask DataFrame**: We start by converting our Pandas DataFrame into a Dask DataFrame to facilitate efficient handling of large datasets.

2. **Prepare Predictor Variables**: We extract elevation and precipitation as predictor variables and convert them to Dask arrays.

3. **Define Target Variable**: NDVI is defined as the target variable, also converted to a Dask array.

4. **Split Data**: The dataset is split into training and testing sets using Dask's train-test split function.

5. **Model Initialization**: A linear regression model from the Dask-ML library is initialized.

6. **Model Fitting**: The model is fitted using the training data.

7. **Predictions and Evaluation**: Predictions are made on the test set, and the model's performance is evaluated using Mean Squared Error (MSE) and R-squared (R²) metrics.

In [ ]:
# Import necessary libraries
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split as dask_train_test_split
from dask_ml.linear_model import LinearRegression as DaskLinearRegression
from dask_ml.metrics import mean_squared_error as dask_mean_squared_error, r2_score as dask_r2_score

ddf = dd.from_pandas(df, npartitions=4)
X_dd = ddf[['elevation', 'precipitation']].to_dask_array(lengths=True)  # Convert to Dask array
y_dd = ddf['NDVI'].to_dask_array(lengths=True)  # Convert to Dask array

X_train_dd, X_test_dd, y_train_dd, y_test_dd = dask_train_test_split(X_dd, y_dd, test_size=0.2, random_state=42)

dask_model = DaskLinearRegression()
dask_model.fit(X_train_dd, y_train_dd)
y_pred_dd = dask_model.predict(X_test_dd)

mse_dd = dask_mean_squared_error(y_test_dd, y_pred_dd)
r2_dd = dask_r2_score(y_test_dd, y_pred_dd)

print(f"Dask Mean Squared Error (MSE): {mse_dd}")
print(f"Dask R-squared (R2 Score): {r2_dd}")


Dask Mean Squared Error (MSE): 0.0033128704621764333
Dask R-squared (R2 Score): 0.1792045995902659
